Creating keywords dictionary from the job description. This dictionary will be used to filter out the resumes suitable for that job. 
For example, if the job description/job post is about Data Analyst,the dictionary will have all the words relevant to Data Analyst, and it will find for those words in the resume and select it. 

In [ ]:
import spacy
import nltk
import random
import pickle
import gensim
import numpy
import pyLDAvis.gensim
import pandas as pd
import matplotlib.pyplot as plt

from spacy.lang.en import English
from spacy import displacy
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models import Phrases
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

%matplotlib inline

/libs/base/past/types/oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


In [ ]:
!pip install pyLDAvis
!pip install paramiko
!pip install gensim

    100% |################################| 1.6MB 5.5MB/s ta 0:00:01
    100% |################################| 235kB 5.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyLDAvis ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/66/0d/3b/484ceb9c1b38c5cb9e0ee9e18ccbf1f1b26d0ff273e91a2892
Successfully built pyLDAvis
  Found existing installation: python-dateutil 2.7.2
    Uninstalling python-dateutil-2.7.2:
      Successfully uninstalled python-dateutil-2.7.2
You are using pip version 9.0.3, however version 20.3.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 215kB 7.1MB/s ta 0:00:011
    100% |################################| 2.7MB 43.8MB/s ta 0:00:01
    100% |################################| 71kB 44.6MB/s ta 0:00:01
    100% |################################| 962kB 53.1MB/s ta 0:00:01
    100% |################################| 409kB 54.3MB/s ta 0:00:01
  Found existing installat

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=PendingDeprecationWarning)
warnings.filterwarnings('ignore', category=ResourceWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
parser = spacy.load('en_core_web_sm')

In [ ]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.is_punct:
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('at')
        elif token.pos_ == "ADJ" or token.pos_ == "VERB" or token.pos_ == "RBR" or token.pos_ == "RBS" or token.pos_ == "RB" or token.pos_ == "RP":
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
from spacy.lang.en import English
from spacy import displacy
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models import Phrases
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

In [ ]:
for w in ['ran', 'happier', 'charging']:
    print(w, get_lemma(w), get_lemma2(w))

ran run ran
happier happy happier
charging charge charging


In [ ]:
en_stop = set(nltk.corpus.stopwords.words('english'))

In [ ]:
import pandas as pd
raw_data = pd.read_csv("/data/ds_test.csv")
raw_data.fillna("", inplace = True)

print(raw_data.head(1))

             crawl_timestamp  \
0  2019-02-06 05:26:22 +0000   

                                                 url  \
0  https://www.indeed.com/viewjob?jk=fd83355c2b23...   

                     job_title            category             company_name  \
0  Enterprise Data Scientist I  Accounting/Finance  Farmers Insurance Group   

             city state country   inferred_city inferred_state  ...  \
0  Woodland Hills    CA     Usa  Woodland hills     California  ...   

                                     job_description   job_type  \
0  Read what people are saying about working here...  Undefined   

  salary_offered job_board  geo        cursor contact_email  \
0                   indeed  usa  1.550000e+15                 

   contact_phone_number                           uniq_id html_job_description  
0                        3b6c6acfcba6135a31c83bd7ea493b18                       

[1 rows x 22 columns]


In [ ]:
def prepare_text(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [ ]:
%%time

job_description_data = []
job_requirement_data = []
required_qualification_data = []

for index, entry in raw_data.iterrows():
  #print(entry)
  job_description_tokens = prepare_text(str(entry["job_description"]).strip())
  #print(job_description_tokens)
  #job_requirement_tokens = prepare_text(str(entry["job_description"]).strip())
  #required_qualification_tokens = prepare_text(str(entry["RequiredQual"]).strip())
  
  if not len(job_description_tokens) == 0:
    job_description_data.append(job_description_tokens)

CPU times: user 8 s, sys: 342 ms, total: 8.34 s
Wall time: 8.63 s


In [ ]:
%%time

job_description_dictionary = corpora.Dictionary(job_description_data)

CPU times: user 27.9 ms, sys: 2.1 ms, total: 30 ms
Wall time: 28.8 ms


In [ ]:
count = 0
print("{} words present in job_description_dictionary.".format(len(job_description_dictionary)))
print("Examples from job_description_dictionary...")
for k, v in job_description_dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

2297 words present in job_description_dictionary.
Examples from job_description_dictionary...
0 02/05/2019
1 absolutely
2 action
3 aggregation
4 also
5 amongst
6 analysis
7 analyst
8 analytics
9 application
10 arima


In [ ]:
%%time

job_description_bow_corpus = [job_description_dictionary.doc2bow(text) for text in job_description_data]

CPU times: user 22.3 ms, sys: 1.03 ms, total: 23.4 ms
Wall time: 22.9 ms


In [ ]:
%%time

job_description_bow_corpus = [job_description_dictionary.doc2bow(text) for text in job_description_data]

CPU times: user 17.2 ms, sys: 2.02 ms, total: 19.3 ms
Wall time: 18.8 ms


In [ ]:
# print("Examples from job_description_bow_corpus...")
# for i in range(len(job_description_bow_corpus[10])):
#   print("Word {} (\"{}\") appears {} time.".format(job_description_bow_corpus[10][i][0], job_description_dictionary[job_description_bow_corpus[10][i][0]], job_description_bow_corpus[10][i][1]))


In [ ]:
# print("Examples from job_description_bow_corpus...")
# for i in range(len(job_description_bow_corpus[10])):
#   print("Word {} (\"{}\") appears {} time.".format(job_description_bow_corpus[10][i][0], job_description_dictionary[job_description_bow_corpus[10][i][0]], job_description_bow_corpus[10][i][1]))

In [ ]:
%%time

job_description_tfidf_model = models.TfidfModel(job_description_bow_corpus)
job_description_tfidf_corpus = job_description_tfidf_model[job_description_bow_corpus]

CPU times: user 11.8 ms, sys: 0 ns, total: 11.8 ms
Wall time: 89.4 ms


In [ ]:
pickle.dump(job_description_bow_corpus, open('job_description_bow_corpus.pkl', 'wb'))
pickle.dump(job_description_tfidf_corpus, open('job_description_tfidf_corpus.pkl', 'wb'))
job_description_dictionary.save('job_description_dictionary.gensim')

In [ ]:
NUM_OF_CORES = 2

In [ ]:
%%time

JOB_DESCRIPTION_NUM_TOPICS = 5
job_description_bow_lda_model = gensim.models.LdaMulticore(job_description_bow_corpus, num_topics = JOB_DESCRIPTION_NUM_TOPICS, id2word=job_description_dictionary, passes=100, workers=NUM_OF_CORES)
job_description_bow_lda_model.save('job_description_bow_lda_model.gensim')
job_description_tfidf_lda_model = gensim.models.LdaMulticore(job_description_tfidf_corpus, num_topics = JOB_DESCRIPTION_NUM_TOPICS, id2word=job_description_dictionary, passes=100, workers=NUM_OF_CORES)
job_description_tfidf_lda_model.save('job_description_tfidf_lda_model.gensim')

CPU times: user 48.9 s, sys: 3.88 s, total: 52.7 s
Wall time: 1min 6s


In [ ]:
# job_description_bow_topics = job_description_bow_lda_model.print_topics(num_words=10)
# print("Job Description Topics (BoW): ")
# for idx, topic in job_description_bow_topics:
#     print('Topic: {} \nWords: {}'.format(idx, topic))

# job_description_tfidf_topics = job_description_tfidf_lda_model.print_topics(num_words=10)
# print("\n\nJob Description Topics (TF-IDF): ")
# for idx, topic in job_description_tfidf_topics:
#     print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
job_description_dictionary = gensim.corpora.Dictionary.load('job_description_dictionary.gensim')
job_description_bow_corpus = pickle.load(open('job_description_bow_corpus.pkl', 'rb'))
job_description_bow_lda_model = gensim.models.ldamodel.LdaModel.load('job_description_bow_lda_model.gensim')
job_description_tfidf_corpus = pickle.load(open('job_description_tfidf_corpus.pkl', 'rb'))
job_description_tfidf_lda_model = gensim.models.ldamodel.LdaModel.load('job_description_tfidf_lda_model.gensim')

In [ ]:
%%time

new_doc = "Data science, machine learning, optimization models, Master’s degree in Machine Learning, Computer Science, Information Technology, Operations Research, Statistics, Applied Mathematics, Econometrics, Successful completion of one or more assessments in Python, Spark, Scala, or R, Using open source frameworks (for example, scikit learn, tensorflow, torch)"
document = parser(new_doc)

print("Noun Phrases...")
print("--- [Format: Noun Phrase -> Root Text] ---")
for noun_phrase in document.noun_chunks:
  #print(noun_phrase.text, noun_phrase.label_, noun_phrase.root.text)
  print(noun_phrase.text + " -> " + noun_phrase.root.text)

new_doc = prepare_text(new_doc)
new_doc_bow = job_description_dictionary.doc2bow(new_doc)
new_doc_tfidf = job_description_tfidf_model[new_doc_bow]

# print("\nTopics relevant to new document are: ")
# print(job_description_bow_lda_model.get_document_topics(new_doc_bow, minimum_probability=0.1))

counter = 0
print("\n\nTopics relevant to new document are (BoW): ")
for index, score in sorted(job_description_bow_lda_model[new_doc_bow], key=lambda tup: -1*tup[1]):
  if counter == 0:
    print("Score: {}\t Topic: {}".format(score, job_description_bow_lda_model.print_topic(index, 10)))
    highest_score = score
    counter = counter + 1
  elif highest_score - score <= 0.3:
    print("Score: {}\t Topic: {}".format(score, job_description_bow_lda_model.print_topic(index, 10)))
    counter = counter + 1
  else:
    break

counter = 0
print("\n\nTopics relevant to new document are (TF-IDF): ")
for index, score in sorted(job_description_tfidf_lda_model[new_doc_tfidf], key=lambda tup: -1*tup[1]):
  if counter == 0:
    print("Score: {}\t Topic: {}".format(score, job_description_tfidf_lda_model.print_topic(index, 10)))
    highest_score = score
    counter = counter + 1
  elif highest_score - score <= 0.3:
    print("Score: {}\t Topic: {}".format(score, job_description_tfidf_lda_model.print_topic(index, 10)))
    counter = counter + 1
  else:
    break

Noun Phrases...
--- [Format: Noun Phrase -> Root Text] ---
Data science -> science
machine learning -> learning
optimization models -> models
Master’s degree -> degree
Machine Learning -> Learning
Computer Science -> Science
Information Technology -> Technology
Operations Research -> Research
Statistics -> Statistics
Applied Mathematics -> Mathematics
Econometrics -> Econometrics
Successful completion -> completion
one or more assessments -> assessments
Python -> Python
Spark -> Spark
Scala -> Scala
R -> R
example -> example
scikit -> scikit


Topics relevant to new document are (BoW): 
Score: 0.8778071403503418	 Topic: 0.052*"data" + 0.035*"experience" + 0.017*"business" + 0.015*"team" + 0.014*"science" + 0.012*"model" + 0.012*"skill" + 0.011*"machine" + 0.010*"years" + 0.009*"analytics"


Topics relevant to new document are (TF-IDF): 
Score: 0.8428539633750916	 Topic: 0.003*"business" + 0.002*"model" + 0.002*"analytics" + 0.002*"customer" + 0.002*"technology" + 0.002*"application" + 

In [ ]:
gensim.corpora.Dictionary.load('job_description_dictionary.gensim')

In [ ]:
gensim.models.ldamodel.LdaModel.load('job_description_bow_lda_model.gensim')

In [ ]:
job_description_dictionary = gensim.corpora.Dictionary.load('job_description_dictionary.gensim')

In [ ]:
new_doc = prepare_text("Data science, machine learning, optimization models, Master’s degree in Machine Learning, Computer Science, Information Technology, Operations Research, Statistics, Applied Mathematics, Econometrics, Successful completion of one or more assessments in Python, Spark, Scala, or R, Using open source frameworks (for example, scikit learn, tensorflow, torch)")

In [ ]:
def prepare_text(text):
	en_stop = set(nltk.corpus.stopwords.words('english'))
	tokens = tokenize(text)
	tokens = [token for token in tokens if len(token) > 3]
	tokens = [token for token in tokens if token not in en_stop]
	tokens = [get_lemma(token) for token in tokens]
	return tokens

In [ ]:
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.is_punct:
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('at')
        elif token.pos_ == "ADJ" or token.pos_ == "VERB" or token.pos_ == "RBR" or token.pos_ == "RBS" or token.pos_ == "RB" or token.pos_ == "RP":
            continue
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [ ]:
parser = spacy.load('en_core_web_sm')

In [ ]:
new_doc_bow = job_description_dictionary.doc2bow(new_doc)

In [ ]:
job_description_bow_lda_model.get_document_topics(new_doc_bow, minimum_probability=0.1)

In [ ]:
job_description_bow_lda_model = gensim.models.ldamodel.LdaModel.load('job_description_bow_lda_model.gensim')